#Dataset

In [15]:
import pickle
import numpy as np
dataset=pickle.load(open('/content/drive/My Drive/KEMOC/dataset_30.pkl', 'rb'), encoding='iso-8859-1')

In [24]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

# LOSO를 위한 데이터셋 저장
LOSO_dataset={}

removed_dataset={}

for ID in dataset.keys():
    if ID=='p22' or ID=='p27' or ID=='p8' or ID=='p26' or ID=='p32':
        continue
    else:
        removed_dataset[ID]={"data":dataset[ID]['data'], "labels":dataset[ID]['labels']}
import torch
from torch.utils.data import TensorDataset, DataLoader

# LOSO를 수행하기 위한 데이터셋 저장
LOSO_dataset={}
for ID in removed_dataset.keys():
    LOSO_dataset[ID]={}
    data=None
    labels=None
    for ID2 in removed_dataset.keys():
        if ID==ID2:
            X=np.concatenate((np.array(removed_dataset[ID]['data']['bvp']).reshape(-1,120,1),np.array(removed_dataset[ID]['data']['eda']).reshape(-1,120,1), np.array(removed_dataset[ID]['data']['temp']).reshape(-1,120,1), np.array(removed_dataset[ID]['data']['ecg']).reshape(-1,120,1)), axis=2)
            Y=removed_dataset[ID]['labels']['self_arousal']
            Y=np.array([item[0] for item in Y])
            X=torch.Tensor(X)
            Y=torch.Tensor(Y).long()
            test_data=TensorDataset(X,Y)
            if 'test' not in LOSO_dataset[ID]:
                LOSO_dataset[ID]['test']=test_data
        elif ID!=ID2:
            X=np.concatenate((np.array(removed_dataset[ID2]['data']['bvp']).reshape(-1,120,1),np.array(removed_dataset[ID2]['data']['eda']).reshape(-1,120,1), np.array(removed_dataset[ID2]['data']['temp']).reshape(-1,120,1), np.array(removed_dataset[ID2]['data']['ecg']).reshape(-1,120,1)), axis=2)
            Y=removed_dataset[ID2]['labels']['self_arousal']
            Y=np.array([item[0] for item in Y])
            X=torch.Tensor(X)
            Y=torch.Tensor(Y).long()
            train_data=TensorDataset(X,Y)
            if 'train' not in LOSO_dataset[ID]:
                LOSO_dataset[ID]['train']={ID2:train_data}
            else:
                LOSO_dataset[ID]['train'][ID2]=train_data


#Pipeline

## Hyper pasrameter

In [ ]:
!pip install learn2learn

In [18]:
import learn2learn as l2l
import argparse
import random

import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
import torchvision

In [19]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 120
input_size = 4
hidden_size = 32
num_layers = 2
num_classes = 2

lr=0.05
maml_lr=0.05 
iterations=500
ways=2
shots=2
tps=16
fas=5

## Model and metric

In [20]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [21]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


In [22]:
training_data.keys()

dict_keys(['p10', 'p11', 'p23', 'p1', 'p15', 'p13', 'p14', 'p19', 'p31', 'p25', 'p4', 'p5', 'p28', 'p24', 'p9'])

# Training

In [ ]:
iterations=500
for ID in LOSO_dataset.keys():
    training_data=LOSO_dataset[ID]['train']
    model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)
    meta_model = l2l.algorithms.MAML(model, lr=maml_lr)

    opt = optim.Adam(meta_model.parameters(), lr=lr)
    loss_func = nn.CrossEntropyLoss()
    with torch.backends.cudnn.flags(enabled=False):

        for iteration in range(iterations):
            iteration_error = 0.0
            iteration_acc = 0.0
            iteration_f1 = 0.0
            for _ in range(tps):
                sample_ID=random.choice(list(training_data.keys()))   
                sample_data=training_data[sample_ID]
                meta_data = l2l.data.MetaDataset(sample_data)
                train_tasks = l2l.data.TaskDataset(meta_data,
                                        task_transforms=[
                                                l2l.data.transforms.NWays(meta_data, ways),
                                                l2l.data.transforms.KShots(meta_data, 2*shots),
                                                l2l.data.transforms.LoadData(meta_data),
                                                l2l.data.transforms.RemapLabels(meta_data),
                                                l2l.data.transforms.ConsecutiveLabels(meta_data),
                                        ],
                                        num_tasks=1)
                learner = meta_model.clone()
                train_task = train_tasks[0]
                data, labels = train_task
                data = data.to(device)
                labels = labels.to(device)

                # Separate data into adaptation/evalutation sets
                adaptation_indices = np.zeros(data.size(0), dtype=bool)
                adaptation_indices[np.arange(shots*ways) * 2] = True
                evaluation_indices = torch.from_numpy(~adaptation_indices)
                adaptation_indices = torch.from_numpy(adaptation_indices)
                adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
                evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

                # Fast Adaptation
                for step in range(fas):
                    train_error = loss_func(learner(adaptation_data), adaptation_labels)
                    learner.adapt(train_error)

                # Compute validation loss
                predictions = learner(evaluation_data)

                valid_error = loss_func(predictions, evaluation_labels)
                valid_error /= len(evaluation_data)
                predictions = predictions.argmax(dim=1)
                valid_accuracy = accuracy_score(predictions.cpu(), evaluation_labels.cpu())
                valid_f1 = f1_score(predictions.cpu(), evaluation_labels.cpu())
                iteration_error += valid_error
                iteration_acc += valid_accuracy
                iteration_f1 += valid_f1

            iteration_error /= tps
            iteration_acc /= tps
            iteration_f1 /= tps
            print('Iteration: {} Loss : {:.3f} Acc : {:.3f} F1 : {:.3f}'.format(iteration, iteration_error.item(), iteration_acc, iteration_f1))

            # Take the meta-learning step
            opt.zero_grad()
            iteration_error.backward()
            opt.step()
    PATH = '/content/drive/My Drive/KEMOC/META_MODEL/AROUSAL/'
    torch.save(meta_model, PATH + 'model_'+ID+'.pt')  

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py:390: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  if param.grad is not None:


Iteration: 0 Loss : 0.173 Acc : 0.391 F1 : 0.506
